# Salt synthetic

An example of creating a simple synthetic salt model. Lets start with a simple plotting function.

# Salt synthetic

An example of creating a simple synthetic salt model. Lets start with a simple plotting function.

In [ ]:
import numpy as np
def createPlot(data,p1,p2,p3):
    if isinstance(data,np.ndarray):
        fld=data
        hyper=Hypercube.hypercube(ns=[data.shape[2],data.shape[1],data.shape[0]])
    else:
        fld=data.getNdArray()
        hyper=data.getHyper()
    ax1=hyper.axes[0]
    ax2=hyper.axes[1]
    ax3=hyper.axes[2]
    mn=fld.min()
    mx=fld.max()
    front=fld[p3,:,::-1].T
    top=fld[::-1,:,p1]
    side=fld[:,p2,::-1].T
    b=(ax1.o,ax1.o+ax1.d*(ax1.n-1),ax2.o,ax2.o+ax2.d*(ax2.n-1))
    frontF=hv.Image(front,kdims=["X","Z"],bounds=(ax2.o,ax1.o,ax2.o+ax2.d*(ax2.n-1),ax1.o+ax1.d*(ax1.n-1))).opts(\
             cmap="gist_rainbow",clim=(mn,mx),toolbar=None,invert_yaxis=True,aspect=1,axiswise=True)
    sideF=hv.Image(side,kdims=["Y","Z"],bounds=(ax3.o,ax1.o,ax3.o+ax3.d*(ax3.n-1),ax1.o+ax1.d*(ax1.n-1))).opts(\
             cmap="gist_rainbow",clim=(mn,mx),toolbar=None,yaxis=None,invert_yaxis=True,aspect=1.,axiswise=True)
    topF=hv.Image(top,kdims=["X","Y"],bounds=(ax2.o,ax2.o,ax2.o+ax2.d*(ax2.n-1),ax3.o+ax3.d*(ax3.n-1))).opts(\
             cmap="gist_rainbow",clim=(mn,mx),toolbar=None,xaxis=None,aspect=1,axiswise=True)
    botF=hv.Layout(frontF+sideF).opts(toolbar=None)
    tot=hv.Layout(topF+hv.Empty()+frontF+sideF).cols(2).opts(toolbar=None)
    return tot
    
    

We will begin by installing a basement at 5500 m/s.

In [ ]:
import syntheticModel
import holoviews as hv
hv.extension("bokeh","matplotlib",logo=False)

hyper=syntheticModel.hypercube(ns=[20,800,800],os=[0,0,0],ds=[10,10,10])
basement=syntheticModel.geoModel(hyper,{"vel":5000.},"vel")

Lets define several different types of deposity functions.

In [ ]:
depositC=syntheticModel.deposit()  #Constant velocity
depositI=syntheticModel.deposit(interbedThick=20.,interbedThickDev=13.,interbedPropDev=100.) #Interbeded layer
depositN=syntheticModel.deposit(noiseDev=300.) #Add some noise to constant velocity
depositIC=syntheticModel.deposit(interbedThick=15.,interbedThickDev=13.,interbedPropDev=100.,interbedDev=4.) #Interbed with noise

Lets begin by putting in a simple deposition layer and look at the current model.

In [ ]:
update=depositIC.apply(basement,thick=80, prop=4100,seed=23)
plt=createPlot(update.get(),40,400,400)
plt

Lets deposit another layer

In [ ]:
update=depositI.apply(update,thick=80, prop=3250,seed=24)
plt=createPlot(update.get(),40,400,400)
plt

In [ ]:
squished=syntheticModel.squish().apply(update,begFreq=70,maxShift=470,seed=44)
depositIC=syntheticModel.deposit(interbedThick=15.,interbedThickDev=13.,interbedPropDev=200.,interbedDev=4.) #Interbed with noise
squished=depositIC.apply(squished,thick=70, prop=2900,seed=46)

plt=createPlot(squished.get(),170,400,400)
plt

In [ ]:
squished2=syntheticModel.squish().apply(update,begFreq=70,maxShift=470,seed=44)
squished2=depositIC.apply(squished2,thick=70, prop=2900,seed=46)

plt=createPlot(squished2.get(),170,400,400)
plt

In [ ]:
dep2=depositIC.apply(squished,thick=40, prop=2700,seed=53)
dep2=depositIC.apply(dep2,thick=40, prop=2400,seed=54)
dep2=depositIC.apply(dep2,thick=40, prop=2200,seed=55)
dep2=syntheticModel.squish().apply(dep2,begFreq=40,maxShift=300,seed=56)
dep2=depositIC.apply(dep2,thick=40, prop=1900.,seed=57)
plt=createPlot(dep2.get(),270,400,400)
plt

In [ ]:
dep3=depositC.apply(dep2,thick=1, prop=1850,seed=58)
np.random.seed(99)
xs=np.random.uniform(low=.1,high=.9,size=(10,10))
ys=np.random.uniform(low=.1,high=.9,size=(10,10))
zs=np.random.uniform(low=.1,high=.9,size=(10,10))
angles=np.random.uniform(low=23,high=30,size=(10,10))
rads=np.random.uniform(low=1850,high=2150,size=(10,10))
shifts=np.random.uniform(low=130,high=160,size=(10,10))
lengths=np.random.uniform(low=300,high=440,size=(10,10))
azimuth=np.random.uniform(low=40,high=45,size=(10,10))
seeds=np.random.randint(99,size=(10,10))



In [ ]:
x=dep2.get().getNdArray()
print(x.min(),x.max())


In [ ]:
dep3=depositC.apply(dep2,thick=1, prop=1850,seed=86)

for i in range(xs.shape[0]):
    for j in range(xs.shape[1]):
        print("working on",j,i)
        dep3=syntheticModel.fault().apply(dep3,angle=angles[i,j],radius=rads[i,j],begx=xs[i,j],begy=ys[i,j],\
                            azimuth=azimuth[i,j],seed=seeds[i,j],\
        begz=zs[i,j],indicateI=True,shift=shifts[i,j],ruptureLength=lengths[i,j])

    dep3=depositC.apply(dep3,thick=1,prop=1850)

    
    
plt=createPlot(dep3.get(),120,340,490)
plt

In [ ]:
plt=createPlot(dep3.getIntField("indicator"),120,340,490)
plt

In [ ]:
mt7=syntheticModel.salt().apply(dep3,seed=238,beg_freq=9.5,applyHills=True,\
            salt_value={"vel":4500})
mt7=depositC.apply(mt7,prop=1450,thick=100)
plt=createPlot(mt7.get(),400,403,725)
plt

In [ ]:
plt=createPlot(mt7.get(),400,403,725)
plt